In [1]:
from ase import Atoms
from ase.calculators.espresso import Espresso, EspressoProfile
from ase.optimize import BFGS
from ase.io import write

## 1. 計算する系の設定（例：シリコン結晶）

In [2]:
# シリコンのダイヤモンド構造
a = 5.43  # 格子定数 (Å)
silicon = Atoms('Si2',
                positions=[(0, 0, 0), (0.25*a, 0.25*a, 0.25*a)],
                cell=[(0, a/2, a/2), (a/2, 0, a/2), (a/2, a/2, 0)],
                pbc=True)

## 2. Quantum ESPRESSO計算の設定
QEはエネルギーの設定がRy (=13.606 eV) 単位であることに注意．

In [3]:
# https://pseudopotentials.quantum-espresso.org/legacy_tables/original-qe-pp-library/si
pseudopotentials = {'Si': 'Si.rel-pbe-rrkj.UPF'}

In [4]:
input_data = {
    'control': {
        'calculation': 'scf',      # 自己無撞着計算
        'restart_mode': 'from_scratch',
        'prefix': 'silicon',
    },
    'system': {
        'ecutwfc': 30.0,           # カットオフエネルギー (Ry)
        'ecutrho': 240.0,          # 電荷密度のカットオフ (Ry)
        'occupations': 'smearing',
        'smearing': 'gaussian',
        'degauss': 0.01,
    },
    'electrons': {
        'conv_thr': 1.0e-8,        # 収束判定
        'mixing_beta': 0.7,
    },
}

kpts = (5, 5, 5)  # k点メッシュ

## 3. Calculatorの設定

In [5]:
calc = Espresso(
    pseudopotentials=pseudopotentials,
    tstress=True,              # 応力を計算
    tprnfor=True,              # 力を計算
    input_data=input_data,
    kpts=kpts,
    profile=EspressoProfile(command='pw.x', pseudo_dir='/root/pseudo_dir')
)

silicon.calc = calc

## 4. エネルギー計算

In [6]:
%%time
try:
    energy = silicon.get_potential_energy()
    forces = silicon.get_forces()
    stress = silicon.get_stress()
    
    print("=" * 50)
    print("計算結果:")
    print("=" * 50)
    print(f"全エネルギー: {energy:.6f} eV")
    print(f"\n原子に働く力 (eV/Å):")
    for i, force in enumerate(forces):
        print(f"  原子 {i+1}: [{force[0]:8.4f}, {force[1]:8.4f}, {force[2]:8.4f}]")
    print(f"\n応力テンソル (eV/Å³):")
    print(f"  {stress}")
    print("=" * 50)
    
    # 結果を保存
    write('silicon_optimized.xyz', silicon)
    print("\n構造を silicon_optimized.xyz に保存しました")
    
except Exception as e:
    print(f"エラーが発生しました: {e}")
    print("\n注意: Quantum ESPRESSOがインストールされ、")
    print("擬ポテンシャルファイルが正しく配置されている必要があります")

計算結果:
全エネルギー: -214.279685 eV

原子に働く力 (eV/Å):
  原子 1: [ -0.0000,   0.0000,  -0.0000]
  原子 2: [  0.0000,  -0.0000,   0.0000]

応力テンソル (eV/Å³):
  [-0.01236758 -0.01236758 -0.01236758 -0.         -0.         -0.        ]

構造を silicon_optimized.xyz に保存しました
CPU times: user 13.6 ms, sys: 0 ns, total: 13.6 ms
Wall time: 1.22 s


In [7]:
with open('espresso.pwo', 'r') as f:
    output = f.read()
print(output)


     Program PWSCF v.7.4.1 starts on  4Nov2025 at 19:45:33 

     This program is part of the open-source Quantum ESPRESSO suite
     for quantum simulation of materials; please cite
         "P. Giannozzi et al., J. Phys.:Condens. Matter 21 395502 (2009);
         "P. Giannozzi et al., J. Phys.:Condens. Matter 29 465901 (2017);
         "P. Giannozzi et al., J. Chem. Phys. 152 154105 (2020);
          URL http://www.quantum-espresso.org", 
     in publications or presentations arising from this work. More details at
     http://www.quantum-espresso.org/quote

     Serial multi-threaded version, running on   12 processor cores
     3405 MiB available memory on the printing compute node when the environment starts
 
     Reading input from espresso.pwi

     Current dimensions of program PWSCF are:
     Max number of different atomic species (ntypx) = 10
     Max number of k-points (npk) =  40000
     Max angular momentum in pseudopotentials (lmaxx) =  4
     Message from routine setup